## Introduction

Every year, Formula One travels all around the world to host races in different countries. Huge numbers of people and lots of equipment have to be hauled from location to location, causing significant environmental impact.

Formula One needs to do everything it can to lessen its carbon emissions, given that it's a spectacle that entirely revolves around motor cars. Some improvements have been made, such as the move to hybrid engines in 2014, but there's a lot more that could be done. 

One particularly simple change would be to organise the season so that the total distance travelled between race weekends is minimised. As I'll show, the current calendar order is very inefficient and involves back-and-forth trips between continents. I will propose an improved ordering with a lower total distance travelled, and hence improved carbon emissions.

It should be noted that this analysis will be based on straight line distances from racetrack to racetrack, and assumes that the same number of people/amount of equipment is being hauled each time. In reality, I imagine that there's a lot of variation in what is hauled, where it's hauled via, and how it's hauled, so in that regard this will be a fairly simplistic exercise. I nonetheless expect it to be indicative of the kind of improvements F1 could make.

Also, this is basically a version of the travelling salesman problem. Even with what seems like a relatively small set of nodes such as an F1 season, all the possible permutations make a brute force approach (i.e. calculating the total distance of every possible route through the point and choosing the shortest) a very heavy calculation. Much smarter minds than mine have come up with complex algorithms to solve this problem - I will create a simplistic approach that comes up with a reasonable answer for a given season.

### Loading and exploring the data

I'll be using the handy [Ergast API](http://ergast.com/mrd/) to get a season's schedule with longitudes and latitudes.

I'll then calculate the distances using GeoPandas, and visualise these journeys using Plotly's mapping functionality.

In [ ]:
import requests
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
from itertools import permutations
from scipy import stats

pd.options.mode.chained_assignment = None

def add_route_distance(gdf):
    gdf["distance_in_km"] = 0
    for i in range(1, len(gdf.index)):
        gdf["distance_in_km"].iloc[i] = gdf["geometry"].iloc[i].distance(gdf["geometry"].iloc[i-1]) / 1000
        
    return gdf

# Function to return a geodataframe with distances for a given season
def get_season_geodataframe(season):

    # Call Ergast API and insert results into a dataframe
    api_url = "https://ergast.com/api/f1/" + str(season) + ".json"

    raw_result = requests.get(api_url).json()
    race_list = raw_result["MRData"]["RaceTable"]["Races"]
    races_df = pd.DataFrame(race_list)
    races_df["round"] = pd.to_numeric(races_df["round"])

    # Extract longitude and latitude from circuit dict
    for coord in ["long", "lat"]:
        races_df[coord] = races_df["Circuit"].apply(lambda x: x["Location"][coord])
        races_df[coord] = pd.to_numeric(races_df[coord])

    # Ignore irrelevant fields
    races_df = races_df[["season", "round", "raceName", "date", "long", "lat"]]

    # Add point geometry data based on longs and lats
    races_gdf = gpd.GeoDataFrame(races_df, geometry=gpd.points_from_xy(races_df["long"], races_df["lat"], crs="EPSG:4326"))
    
    # Convert projection to UTM to allow distance calculation in metres
    races_gdf.to_crs(epsg=3310, inplace=True)

    # Calculate distances to each race from its preceding event
    races_gdf = add_route_distance(races_gdf)
    
    return races_gdf


# Example of function output based on this season
get_season_geodataframe(2022)

In [ ]:
# Plot this season's route around the globe
season = 2022

gdf = get_season_geodataframe(season)

total_distance = int(gdf["distance_in_km"].sum())

fig = go.Figure()

fig.add_trace(
    go.Scattergeo(
        lon=gdf["long"],
        lat=gdf["lat"],
        mode="lines+markers",
        text=gdf["raceName"]
    )
)

fig.update_layout(
    geo_projection_type="orthographic",
    title="F1 " + str(season) + " calendar, total distance: " + str(total_distance) + "km. Drag to rotate."
)


We can clearly see from the above plot that there are some unnecessarily long trips between events, e.g. repeated trips across the Atlantic to get to Miami and Montreal in between European races.

### Finding the shortest route across a season

Next I'm going to write a function to find the shortest route between all races in a given season.

As mentioned in the introduction, a brute force approach to this calculation is not practical. My approach will be to break the nodes into smaller groups, and then iterate through different route orders both between and within those groups.

I will recursively cluster races together if they're within a certain distance of eachother. I will iterate through reducing distances to find the one with the lowest number of permutations at any level.

In [ ]:
def create_distance_matrix(gdf, point_identifier):
    distance_frames = []
    for i in range(0, len(gdf.index)):
        gdf_copy = gdf.copy()
        gdf_copy["distance_in_km"] = gdf_copy["geometry"].distance(gdf["geometry"].iloc[i]) / 1000
        gdf_copy.drop(i, inplace=True)
        gdf_copy[point_identifier + "From"] = gdf[point_identifier].iloc[i]
        gdf_copy.rename(columns={point_identifier:point_identifier + "To"}, inplace=True)
        distance_frames.append(gdf_copy[[point_identifier + "From", point_identifier + "To", "distance_in_km"]])

    distance_matrix = pd.concat(distance_frames)
    distance_matrix.reset_index(inplace=True, drop=True)
    
    return distance_matrix


def create_groups(season_gdf, distance_threshold):

    # Get matrix of distances between all points
    distance_matrix = create_distance_matrix(season_gdf, "round")

    # Loop through races to create groups
    season_gdf["group_id"] = -1
    group_id = 0

    def rounds_in_distance(rounds):
        df = distance_matrix[(distance_matrix["roundFrom"].isin(rounds)) & (~distance_matrix["roundTo"].isin(rounds)) & (distance_matrix["distance_in_km"] <= distance_threshold)]
        return list(dict.fromkeys(list(df["roundTo"])))

    for round in range(1, len(season_gdf.index) + 1):
        if season_gdf[(season_gdf["round"] == round)]["group_id"].iloc[0] != -1:
            continue
        else:
            grouped_rounds = []
            rounds_to_group = [round]

            while len(rounds_to_group) > 0:
                grouped_rounds.extend(rounds_to_group)
                rounds_to_group = rounds_in_distance(grouped_rounds)

            season_gdf["group_id"] = season_gdf.apply(lambda x: group_id if x["round"] in grouped_rounds else x["group_id"], axis=1)

            group_id = group_id + 1


    groups_df = season_gdf.groupby("group_id").agg(
        long=("long", "mean"),
        lat=("lat", "mean")
    ).reset_index()
    
    groups_gdf = gpd.GeoDataFrame(groups_df, geometry=gpd.points_from_xy(groups_df["long"], groups_df["lat"], crs="EPSG:4326"))
    groups_gdf.to_crs(epsg=3310, inplace=True)
    
    group_count = len(groups_gdf.index)
    
    races_per_group = season_gdf.groupby("group_id").round.count().reset_index()
    max_races_per_group = races_per_group["round"].max()
    
    max_permutations = max(group_count, max_races_per_group)
    
    return season_gdf, groups_gdf, max_permutations


def find_best_grouping(season_gdf):
    
    distance_gdfs = {}
    distance_permutations = []
    
    for distance in range(2000, 0, -100):
        season_gdf, groups_gdf, max_permutations = create_groups(season_gdf, distance)
        distance_gdfs[distance] = (season_gdf.copy(), groups_gdf.copy())
        distance_permutations.append((distance, max_permutations))
        
    distance_permutations = pd.DataFrame(distance_permutations, columns=["distance", "permutations"])
    min_permutations = distance_permutations["permutations"].min()
    best_distance = distance_permutations[(distance_permutations["permutations"] == min_permutations)]["distance"].values[0]
    
    return distance_gdfs[best_distance]
    

season_gdf = get_season_geodataframe(2022)
season_gdf, groups_gdf = find_best_grouping(season_gdf)


# Visualise groupings
fig = go.Figure()

fig.add_trace(
    go.Scattergeo(
        lon=season_gdf["long"],
        lat=season_gdf["lat"],
        mode="markers",
        text=season_gdf["group_id"],
        marker_color=season_gdf["group_id"]
  )
)

fig.update_layout(
    geo_projection_type="mercator",
    title="Race Grouping"
)


In [ ]:
def find_best_route_through_points(gdf, point_identifier):
    
    if len(gdf.index) == 1:
        gdf_copy = gdf.copy()
        gdf_copy["iteration"] = 0
        gdf_copy["sorter"] = 0
        
        return gdf_copy

    # Create distance matrix
    distance_matrix = create_distance_matrix(gdf, point_identifier)

    # Use points furthest apart as start & end
    distance_matrix.sort_values("distance_in_km", ascending=False, inplace=True)
    distance_matrix.reset_index(inplace=True)
    starting_point = distance_matrix[point_identifier + "From"].iloc[0]
    ending_point = distance_matrix[point_identifier + "To"].iloc[0]

    # Get all order permutations with fixed start and end points
    def filter_permutations(permutations):
        if permutations[0] == starting_point and permutations[-1] == ending_point:
            return True
        else:
            return False

    points = list(gdf[point_identifier])
    point_perms = list(permutations(points, len(points)))
    point_perms = list(filter(filter_permutations, point_perms))

    # Calculated distances for each permutation
    frames_to_concat = []
    for i in range(0, len(point_perms)):
        gdf_copy = gdf.copy()
        gdf_copy["iteration"] = i
        gdf_copy["sorter"] = gdf_copy[point_identifier].apply(lambda x: point_perms[i].index(x))
        gdf_copy.sort_values("sorter", inplace=True)
        gdf_copy.reset_index(inplace=True, drop=True)

        gdf_copy = add_route_distance(gdf_copy)

        frames_to_concat.append(gdf_copy)

    iterations = pd.concat(frames_to_concat)
    total_distances = iterations.groupby("iteration").distance_in_km.sum().reset_index()
    total_distances.sort_values("distance_in_km", inplace=True)
    total_distances.reset_index(inplace=True)

    best_iteration = total_distances["iteration"].iloc[0]

    return iterations[(iterations["iteration"] == best_iteration)]


def find_optimal_route(groups_gdf, season_gdf):

    # Get best route through groups
    groups_route = find_best_route_through_points(groups_gdf, "group_id")

    # Get best route inside each group
    group_route_dict = {}

    for group in range(0, groups_gdf["group_id"].max() + 1):

        group_races = season_gdf[(season_gdf["group_id"] == group)]
        group_races["race_id"] = range(0, len(group_races.index))
        group_races.reset_index(inplace=True, drop=True)

        group_route = find_best_route_through_points(group_races, "race_id")

        group_route_dict[group] = group_route


    # Reverse order of races in group if needed
    for group_id, races in group_route_dict.items():

        if len(races.index) == 1:
            continue
        else:
            group_sorter = groups_route[(groups_route["group_id"] == group_id)]["sorter"].values[0]
            
            # Distance per race to previous and following group
            if group_sorter == 0:
                races["distance_to_previous_group"] = 0
            else:
                previous_group_point = groups_route[(groups_route["sorter"] == group_sorter - 1)]["geometry"].values[0]
                races["distance_to_previous_group"] = races["geometry"].distance(previous_group_point) / 1000
                
            if group_sorter == groups_route["sorter"].max():
                races["distance_to_next_group"] = 0
            else:
                next_group_point = groups_route[(groups_route["sorter"] == group_sorter + 1)]["geometry"].values[0]
                races["distance_to_next_group"] = races["geometry"].distance(next_group_point) / 1000
                
            # Calculate which ordering results in shortest inter-group distance, then reverse order if needed
            current_distance = races["distance_to_previous_group"].iloc[0] + races["distance_to_next_group"].iloc[-1]
            reverse_distance = races["distance_to_previous_group"].iloc[-1] + races["distance_to_next_group"].iloc[0]
            
            if reverse_distance < current_distance:
                races.sort_values("sorter", ascending=False, inplace=True)
                races.reset_index(inplace=True, drop=True)
                
            races.drop(["distance_to_previous_group", "distance_to_next_group"], axis=1, inplace=True)

    # Concatenate races in correct order
    route_frames = []

    for i in range(0, len(groups_route.index)):
        group_id = groups_route["group_id"].iloc[i]
        route_frames.append(group_route_dict[group_id])

    route = pd.concat(route_frames)
    route.drop(["race_id", "iteration", "sorter"], axis=1, inplace=True)
    route.reset_index(inplace=True, drop=True)
    route = add_route_distance(route)

    return route



# Get optimal route for this season and compare the total distance to the current schedule

optimal_route = find_optimal_route(groups_gdf, season_gdf)
current_route = get_season_geodataframe(2022)

optimal_distance = optimal_route["distance_in_km"].sum()
current_distance = current_route["distance_in_km"].sum()

distance_percentage = "{0:.2%}".format(optimal_distance / current_distance)

fig = go.Figure()

fig.add_trace(
    go.Scattergeo(
        lon=optimal_route["long"],
        lat=optimal_route["lat"],
        mode="lines+markers",
        marker_color=optimal_route["group_id"]
    )
)

fig.update_layout(
    geo_projection_type="orthographic",
    title="Optimal route, " + distance_percentage + " total distance vs. current calendar"
)

fig.show()

Resequencing the 2022 F1 calendar cuts the total journey distance over the season by more than half!

Note how the route crosses over itself on the way from Miami to Montreal. This is a result of having to compromise and process groups in isolation; the processing cost is simply prohibitive without doing so.

### Running the analysis for multiple seasons

It would be interesting to see whether the reduction in journey distance we see for 2022 is typical of all F1 seasons - perhaps 2022 has been a particularly badly-planned calendar in this regard?

I can use the functions I've already defined to run multiple seasons through this method and plot the results.

In [ ]:
# Run through seasons and capture the actual/optimal routes

results = []
gdfs = {}

for season in range(1990, 1996):
    
    season_gdf = get_season_geodataframe(season)
    actual_distance = int(season_gdf["distance_in_km"].sum())
    
    season_gdf, groups_gdf = find_best_grouping(season_gdf)
    
    optimal_route = find_optimal_route(groups_gdf, season_gdf)
    optimal_distance = int(optimal_route["distance_in_km"].sum())
    
    distance_percentage = optimal_distance / actual_distance
    distance_percentage_string = "{0:.2%}".format(distance_percentage)
    
    results.append((season, actual_distance, optimal_distance, distance_percentage, distance_percentage_string))
    gdfs[season] = (season_gdf, optimal_route)
    

In [ ]:
results_df = pd.DataFrame(results, columns=["season", "actual_distance", "optimal_distance", "distance_percentage", "distance_percentage_string"])
results_df["label"] = results_df.apply(lambda x: str(x["season"]) + ": " + x["distance_percentage_string"], axis=1)

mean_percentage = results_df["distance_percentage"].mean()
total_percentage = results_df["optimal_distance"].sum() / results_df["actual_distance"].sum()
readout_string = "Average route length per season: " + "{0:.2%}".format(mean_percentage) + ", total: " + "{0:.2%}".format(total_percentage) 


# Scatter plot

slope, intercept, r, p, std = stats.linregress(results_df["actual_distance"], results_df["optimal_distance"])
fit_line = slope * results_df["actual_distance"] + intercept

axis_min = results_df["optimal_distance"].min() * 0.95
axis_max = results_df["actual_distance"].max() * 1.05

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=results_df["actual_distance"],
        y=results_df["optimal_distance"],
        mode="markers",
        marker_color=results_df["distance_percentage"],
        text = results_df["label"]
    )
)

fig.add_trace(
    go.Scatter(
        x=results_df["actual_distance"],
        y=fit_line,
        mode="lines",
        line_color="#AEB6BF"
    )
)

fig.update_xaxes(
    range=[axis_min, axis_max],
    title="Actual distance (km)"
)
fig.update_yaxes(
    range=[axis_min, axis_max],
    title="Optimised distance (km)"
)

fig.update_layout(
    width=600,
    height=600,
    title="Actual distance vs. optimised distance per season",
    showlegend=False
)

fig.show()

print(readout_string)


***Write about seasons***

In [ ]:
def plot_globe(season, season_gdf, optimal_route):
    
    percentage = optimal_route["distance_in_km"].sum() / season_gdf["distance_in_km"].sum()
    percentage_string = "{0:.2%}".format(percentage)
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scattergeo(
            lon=season_gdf["long"],
            lat=season_gdf["lat"],
            mode="lines",
            line_color="#FF5733",
            name="Actual"
        )
    )
    
    fig.add_trace(
        go.Scattergeo(
            lon=optimal_route["long"],
            lat=optimal_route["lat"],
            mode="lines",
            line_color="#6495ED",
            name="Optimised"
        )
    )
    
    fig.update_layout(
        geo_projection_type="orthographic",
        title="F1 " + str(season) + " season. Optimised distance " + percentage_string + " of actual."
    )
    
    fig.show()
    
# Return map of chosen season vs. optimised route
season = 1990

season_gdf, optimal_route = gdfs[1990]
plot_globe(season, season_gdf, optimal_route)

